In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import os
from sklearn.metrics         import confusion_matrix, classification_report
from sklearn.preprocessing   import StandardScaler
from sklearn.model_selection import train_test_split

path = '/data/home/zhaocj/spn_forcast/df数据整理/../7mer_output/RBIND_7mer'
#path = '/data/home/zhaocj/spn_forcast/df数据整理/../11mer_output/RBIND_11mer'
#path = '../9mer_output/RBIND_9mer'

# test.txt 是X92到X99这几株菌的合并
#path = '../11mer_output/test.txt'

# test2 是每株菌的head
#path = '../11mer_output/test2.txt'

# test3 只是100这株菌
#path = '../11mer_output/test3.txt'

# test4 是每株菌的head, 整理后的数据
#path = '../11mer_output/test4.txt'

# test5 是整理后的数据，是准确的
#path = '../11mer_output/test5.sh'

drug = 'PEN_NM'
drug = 'PEN_MIC'

try:
    df = pd.read_table(path, header=None, names=['SEQ', 'COUNT', 'ID'])
    id_number = pd.read_table('/data/home/zhaocj/spn_forcast/df数据整理/ID.txt')
    ris = pd.read_table('/data/home/zhaocj/spn_forcast/df数据整理/ris.txt')
    mic = pd.read_table('/data/home/zhaocj/spn_forcast/df数据整理/mic.txt')
except:
    print('数据导入失败！/(ㄒoㄒ)/~~')
else:
    print('数据导入成功😄！')
    

/data/build/miniconda3/envs/cenote-taker2_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


数据导入成功😄！


In [2]:
try:
    df = df.pivot_table(index=['ID'], columns='SEQ', values='COUNT', fill_value=0).reset_index()
    
    # 把 df 的ID转换为字符型
    for i in range(df.shape[0]):
        df.iloc[i, 0] = str(df.iloc[i, 0])

    # 选取id_number数据表里有的菌
    df = df.set_index('ID')
    df = df.loc[list(id_number['ID'])].reset_index()
    
    
    # 如果drug里面有MIC则合并MIC的表
    if 'MIC' in drug:
        mic = mic[['ID', drug]]
        for i in range(mic.shape[0]):
            mic.iloc[i, 1] = str(mic.iloc[i, 1])
        df = pd.merge(df, mic, how = 'left', on=['ID'])
    else:
        ris = ris[['ID', drug]]
        df = pd.merge(df, ris, how = 'left', on=["ID"])
    ID = df.iloc[:,0]
    df = df.iloc[:,1:]
    
    #print(df.head())
except:
    print('出错了，哪里错了，不知道。')
else:
    print('运气真好，做对了😄！')

运气真好，做对了😄！


In [3]:
# Create arrays for the features and the target: X, y
X, y = df.iloc[:,:-1], df.iloc[:,-1]

# Create the DMatrix: housing_dmatrix
kmer_dmatrix = xgb.DMatrix(data=X, label=y)

In [6]:
# Create the parameter dictionary for each tree: params 
params = {"objective":"binary:logistic", "max_depth":3}

# Create list of number of boosting rounds
num_rounds = [5, 10, 15]

# Empty list to store final round rmse per XGBoost model
final_rmse_per_round = []


In [8]:

# Iterate over num_rounds and build one model per num_boost_round parameter
for curr_num_rounds in num_rounds:
    # Perform cross-validation: cv_results
    cv_results = xgb.cv(dtrain=kmer_dmatrix, params=params, nfold=3, num_boost_round=curr_num_rounds, metrics="rmse", as_pandas=True, seed=123)
    
    # Append final round RMSE
    #final_rmse_per_round.append(cv_results["test-rmse-mean"].tail().values[-1])
    print(cv_results)

# Print the resultant DataFrame
#num_rounds_rmses = list(zip(num_rounds, final_rmse_per_round))
#print(pd.DataFrame(num_rounds_rmses,columns=["num_boosting_rounds","rmse"]))

XGBoostError: [14:30:27] ../src/objective/regression_obj.cu:120: label must be in [0,1] for logistic regression
Stack trace:
  [bt] (0) /data/build/miniconda3/envs/cenote-taker2_env/lib/python3.6/site-packages/xgboost/lib/libxgboost.so(+0x455ac9) [0x2b39518e7ac9]
  [bt] (1) /data/build/miniconda3/envs/cenote-taker2_env/lib/python3.6/site-packages/xgboost/lib/libxgboost.so(+0x46a0b0) [0x2b39518fc0b0]
  [bt] (2) /data/build/miniconda3/envs/cenote-taker2_env/lib/python3.6/site-packages/xgboost/lib/libxgboost.so(+0x1b4990) [0x2b3951646990]
  [bt] (3) /data/build/miniconda3/envs/cenote-taker2_env/lib/python3.6/site-packages/xgboost/lib/libxgboost.so(XGBoosterUpdateOneIter+0x68) [0x2b395152b4f8]
  [bt] (4) /data/build/miniconda3/envs/cenote-taker2_env/lib/python3.6/lib-dynload/../../libffi.so.7(+0x69dd) [0x2b38a5d189dd]
  [bt] (5) /data/build/miniconda3/envs/cenote-taker2_env/lib/python3.6/lib-dynload/../../libffi.so.7(+0x6067) [0x2b38a5d18067]
  [bt] (6) /data/build/miniconda3/envs/cenote-t

In [ ]:
# Create arrays for the features and the target: X, y
X, y = df.iloc[:,:-1], df.iloc[:,-1]

# Create the training and test sets
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=123)

# 正态转换
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Instantiate the XGBClassifier: xg_cl
xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=10, seed=123)

# Fit the classifier to the training set
xg_cl.fit(X_train_scaled, y_train)

# Predict the labels of the test set: preds
preds = xg_cl.predict(X_test_scaled)

# Compute the accuracy: accuracy
accuracy = float(np.sum(preds==y_test))/y_test.shape[0]

print("accuracy: %f" % (accuracy))

In [ ]:
# Measuring accuracy
# Create the parameter dictionary: params
params = {"objective":"binary:logistic", "max_depth":3}

# Perform cross-validation: cv_results
cv_results = xgb.cv(dtrain=kmer_dmatrix, params=params, 
                    nfold=3, num_boost_round=5, 
                    metrics="error", as_pandas=True, seed=123)